In [17]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
%autoreload 2

In [19]:
%matplotlib inline
import sys
sys.path.append('../')
import RNA 
from utils.sequence_utils import generate_random_mutant
from utils.model_architectures import Linear, NLNN, CNNa
from models.Noisy_models.Neural_network_models import NN_model
from models.Ground_truth_oracles.RNA_landscape_models import RNA_landscape_constructor
from models.Noisy_models.Ensemble import Ensemble_models
from evaluators.Evaluator import Evaluator
from models.Ground_truth_oracles.TF_binding_landscape_models import *

LANDSCAPE_TYPES ={"RNA": [0, 1], 'TF':['SIX6_REF_R1']} # for testing 
LANDSCAPE_TYPES_RNA = {"RNA" : [0,1,12,20,25,31], "TF": []}
LANDSCAPE_TYPES_TF = {"RNA": [], 
                      "TF": ['POU3F4_REF_R1','PAX3_G48R_R1','SIX6_REF_R1', 'VAX2_REF_R1', 'VSX1_REF_R1']}
mutation_rate = 0.5
RNA_alphabet = "UCGA" 
DNA_alphabet = "TCGA"
landscape_constructor = RNA_landscape_constructor()
#load a pre-specified landscapes  (here, index 0), you can also specify to just take a few of them
landscape_constructor.load_landscapes("../data/RNA_landscapes/RNA_landscape_config.yaml", landscapes_to_test=[0])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object=next(landscape_generator)
wt = landscape_object['starting_seqs']['startRNAL14_3']
landscape_oracle = landscape_object['landscape_oracle']

training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


B1L14RNA1 loaded


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


## DQN Explorer

In [8]:
from explorers.dqn_explorer import DQN_Explorer 

num_of_rounds = 5

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=RNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(train_epochs=10)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.14554841726711917
round: 1, cost: 1, evals: 0, top: 0.14554841726711917
round: 2, cost: 1, evals: 0, top: 0.14554841726711917
round: 3, cost: 1, evals: 0, top: 0.14554841726711917
round: 4, cost: 1, evals: 0, top: 0.14554841726711917


In [15]:
landscape_constructor=TF_binding_landscape_constructor()
landscape_constructor.load_landscapes(landscapes_to_test = ['SIX6_REF_R1'])
landscape_generator = landscape_constructor.generate_from_loaded_landscapes()
landscape_object = next(landscape_generator)
wt = landscape_object['starting_seqs']['TF0']
landscape_oracle = landscape_object['landscape_oracle']

# reset model if it already exists 
training_sequences = [wt]
cnn_architecture=CNNa(len(wt), alphabet=DNA_alphabet)
noisy_nn_model=NN_model(landscape_oracle, cnn_architecture)
noisy_nn_model.reset()
noisy_nn_model.update_model(training_sequences)

# evaluate DQN Explorer 
dqn_explorer = DQN_Explorer(alphabet=DNA_alphabet, train_epochs=10, virtual_screen=20)
dqn_explorer.debug = True
dqn_explorer.set_model(noisy_nn_model)
dqn_explorer.run(num_of_rounds, verbose=True)

1 TF landscapes loaded.


/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


round: 0, cost: 1, evals: 0, top: 0.6781429101554447
round: 1, cost: 1, evals: 0, top: 0.6781429101554447
round: 2, cost: 1, evals: 0, top: 0.6781429101554447
round: 3, cost: 1, evals: 0, top: 0.6781429101554447
round: 4, cost: 1, evals: 0, top: 0.6781429101554447


In [20]:
from explorers.dqn_explorer import DQN_Explorer 

dqn_explorer_prod = DQN_Explorer(train_epochs=10)
dqn_explorer_prod.debug = False

In [21]:
# run all evaluations 
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.adaptivity, num_starts=5)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.efficiency, num_starts=5)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.scalability, num_starts=5)

loading landscapes RNA: [0, 1, 12, 20, 25, 31], TF:[]
B1L14RNA1 loaded
B1L50RNA1 loaded
B2L14RNA1+2 loaded
B2L50RNA1+4 loaded
B2L100RNA1+3 loaded
C20B2L100RNA1+3 loaded
0 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


start seq startRNAL14_1
start seq startRNAL14_2
Running on B1L50RNA1
start seq startRNAL50_0
start seq startRNAL50_1
start seq startRNAL50_2
Running on B2L14RNA1+2
start seq startRNAL14_0
start seq startRNAL14_1
start seq startRNAL14_2
Running on B2L50RNA1+4
start seq startRNAL50_0
start seq startRNAL50_1
start seq startRNAL50_2
Running on B2L100RNA1+3
start seq startRNAL100_0
start seq startRNAL100_1
start seq startRNAL100_2
Running on C20B2L100RNA1+3
start seq startRNAL100_0
start seq startRNAL100_1
start seq startRNAL100_2
loading landscapes RNA: [], TF:['POU3F4_REF_R1', 'PAX3_G48R_R1', 'SIX6_REF_R1', 'VAX2_REF_R1', 'VSX1_REF_R1']
5 TF landscapes loaded.
loading complete
Running on POU3F4_REF_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on PAX3_G48R_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on SIX6_REF_R1
start seq TF0
start seq TF1
start seq TF2
start seq TF3
start seq TF4
Running on VAX2_REF_R1
start seq TF

POU3F4_REF_R1,1000,100,108.33840560913086

start seq TF2
POU3F4_REF_R1,100,1,0.18558120727539062

POU3F4_REF_R1,100,10,1.1254701614379883

POU3F4_REF_R1,100,100,10.183694839477539

POU3F4_REF_R1,1000,1,1.990445852279663

POU3F4_REF_R1,1000,10,10.798193216323853

POU3F4_REF_R1,1000,100,108.69755554199219

start seq TF3
POU3F4_REF_R1,100,1,0.15920329093933105

POU3F4_REF_R1,100,10,1.101102590560913

POU3F4_REF_R1,100,100,9.955430746078491

POU3F4_REF_R1,1000,1,1.496490240097046

POU3F4_REF_R1,1000,10,10.909098148345947

POU3F4_REF_R1,1000,100,99.83661842346191

start seq TF4
POU3F4_REF_R1,100,1,0.2188739776611328

POU3F4_REF_R1,100,10,1.1076173782348633

POU3F4_REF_R1,100,100,10.893426895141602

POU3F4_REF_R1,1000,1,1.9809165000915527

POU3F4_REF_R1,1000,10,10.936813354492188

POU3F4_REF_R1,1000,100,100.02221059799194

Running on PAX3_G48R_R1
start seq TF0
PAX3_G48R_R1,100,1,0.22397160530090332

PAX3_G48R_R1,100,10,1.1039106845855713

PAX3_G48R_R1,100,100,10.950556516647339

PAX3_G48R_R1

In [ ]:
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=3)
evaluator_dqn=Evaluator(dqn_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_dqn.evaluate_for_landscapes(evaluator_dqn.consistency_robustness_independence, num_starts=5)

## BO Explorer

In [ ]:
%autoreload 2
from explorers.bo_explorer import BO_Explorer

num_of_rounds = 5
arch1 = NLNN(len(wt), hidden_dims=10, epochs=20)
arch2 = NLNN(len(wt), hidden_dims=20, epochs=20)
arch3 = NLNN(len(wt), hidden_dims=30, epochs=20)
arch4 = NLNN(len(wt), hidden_dims=40, epochs=20)
# ensemble we will use for BO Explorer
nn_model_1 = NN_model(landscape_oracle, arch1)
nn_model_2 = NN_model(landscape_oracle, arch2)
nn_model_3 = NN_model(landscape_oracle, arch3)
nn_model_4 = NN_model(landscape_oracle, arch4)
models = [nn_model_1, nn_model_2, nn_model_3, nn_model_4]
ensemble_model = Ensemble_models(models)
ensemble_model.update_model(training_sequences)

In [ ]:
# evaluate BO Explorer 
bo_explorer = BO_Explorer()
bo_explorer.debug = True
bo_explorer.set_model(ensemble_model)
bo_explorer.run(num_of_rounds, verbose=True)

In [27]:
from explorers.bo_explorer import BO_Explorer

bo_explorer_prod = BO_Explorer()
bo_explorer_prod.debug = False

In [ ]:
# run all evaluations 
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.adaptivity, num_starts=5)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.efficiency, num_starts=5)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.scalability, num_starts=5)

In [ ]:
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_RNA, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=3)
evaluator_bo=Evaluator(bo_explorer_prod,landscape_types=LANDSCAPE_TYPES_TF, path="../simulations/eval/")
evaluator_bo.evaluate_for_landscapes(evaluator_bo.consistency_robustness_independence, num_starts=5)

loading landscapes RNA: [0, 1, 12, 20, 25, 31], TF:[]
B1L14RNA1 loaded
B1L50RNA1 loaded
B2L14RNA1+2 loaded
B2L50RNA1+4 loaded
B2L100RNA1+3 loaded
C20B2L100RNA1+3 loaded


../models/Ground_truth_oracles/RNA_landscape_models.py:21: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  self.loaded_landscapes = yaml.load(cfgfile)


0 TF landscapes loaded.
loading complete
Running on B1L14RNA1
start seq startRNAL14_0
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 101, evals: 10100, top: 0.5464930837468063
round: 2, cost: 200, evals: 20199, top: 0.7689350069465327
round: 3, cost: 388, evals: 30297, top: 0.7963969714803375
round: 4, cost: 488, evals: 40396, top: 0.7963969714803375
round: 5, cost: 588, evals: 50496, top: 0.7963969714803375
round: 6, cost: 687, evals: 60595, top: 0.7963969714803375
round: 7, cost: 786, evals: 70694, top: 0.8540671074772356
round: 8, cost: 884, evals: 80792, top: 0.8540671074772356
round: 9, cost: 982, evals: 90890, top: 0.8540671074772356
round: 0, cost: 1, evals: 0, top: 0.5437468768175177
round: 1, cost: 101, evals: 10099, top: 0.5437468768175177
round: 2, cost: 201, evals: 20198, top: 0.6261327704189319
round: 3, cost: 301, evals: 30298, top: 0.8018893853389146
round: 4, cost: 400, evals: 40397, top: 0.8018893853389146
round: 5, cost: 589, evals: 50496, top: 0

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.5437468768175177
[0.0059023989261968324, 0.9881952021476064, 0.0059023989261968324]
round: 1, cost: 101, evals: 9501.0, top: 0.631625184277509
[0.008661686786836806, 0.9826766264263264, 0.008661686786836806]
round: 2, cost: 201, evals: 18994.0, top: 0.6481023734737
[0.4568860412590248, 0.11682169878219226, 0.42629225995878295]
round: 3, cost: 301, evals: 28557.0, top: 0.887021433490077
[0.7860018539686823, 0.2066826949830249, 0.007315451048292861]
round: 4, cost: 401, evals: 38029.0, top: 0.887021433490077
[0.9595978993521977, 0.020201050323901102, 0.020201050323901102]
round: 5, cost: 501, evals: 47511.0, top: 0.887021433490077
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 6, cost: 601, evals: 56977.0, top: 0.887021433490077
[0.4203503424218783, 0.0029375278387231175, 0.5767121297393986]
round: 7, cost: 701, evals: 66510.0, top: 0.887021433490077
[0.0059196669966989

...

[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.44830885623659755
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 101, evals: 9521.0, top: 0.5860613620888847
[0.40249894634020866, 0.5944896391948751, 0.0030114144649162487]
round: 2, cost: 201, evals: 19044.0, top: 0.5860613620888847
[0.003858962234498953, 0.9922820755310021, 0.003858962234498953]
round: 3, cost: 301, evals: 28538.0, top: 0.5860613620888847
[0.22361256934322482, 0.664958887909802, 0.11142854274697321]
round: 4, cost: 401, evals: 38051.0, top: 0.5860613620888847
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 5, cost: 501, evals: 47532.0, top: 0.5860613620888847
[0.537399875186313, 0.16179138294573248, 0.30080874186795453]
round: 6, cost: 601, evals: 57045.0, top: 0.5860613620888847
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 7, cost: 701, evals: 66518.0, top: 0.5860613620888847
[0.23467254902986598

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.4110586524455687
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 101, evals: 9519.0, top: 0.4110586524455687
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 201, evals: 18986.0, top: 0.6187614745822908
[0.45579324558830864, 0.001893744019788413, 0.5423130103919029]
round: 3, cost: 301, evals: 28468.0, top: 0.6187614745822908
[0.9797499082234402, 0.010125045888279859, 0.010125045888279859]
round: 4, cost: 401, evals: 37950.0, top: 0.6187614745822908
[0.4903090226829521, 0.17282127056789948, 0.3368697067491483]
round: 5, cost: 501, evals: 47430.0, top: 0.6187614745822908
[0.6539314404622601, 0.34364476584352055, 0.0024237936942192923]
round: 6, cost: 601, evals: 56919.0, top: 0.6187614745822908
[0.014800890841592947, 0.005893117220834382, 0.9793059919375726]
round: 7, cost: 701, evals: 66400.0, top: 0.6187614745822908
[0.875025159637

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_regression.py:582: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 0, cost: 1, evals: 0.0, top: 0.07772254319006876
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 1, cost: 101, evals: 9486.0, top: 0.503650775661968
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 2, cost: 201, evals: 19008.0, top: 0.503650775661968
[0.3333333333333333, 0.3333333333333333, 0.3333333333333333]
round: 3, cost: 301, evals: 28516.0, top: 0.503650775661968
[0.11754450193724811, 0.005449920103510259, 0.8770055779592416]
round: 4, cost: 401, evals: 37988.0, top: 0.503650775661968
[0.004706670081583341, 0.9905866598368334, 0.004706670081583341]
round: 5, cost: 501, evals: 47459.0, top: 0.503650775661968
[0.8402423157577436, 0.15745616445198055, 0.0023015197902759225]
round: 6, cost: 601, evals: 56945.0, top: 0.503650775661968
[0.9887634984598083, 0.005618250770095888, 0.005618250770095888]
round: 7, cost: 701, evals: 66474.0, top: 0.503650775661968
[0.7090185310733909, 0